# Metaprogramming

A "meta" program is a program that manipulates programs.

Most common use refers to a program that *generates* another program.

In the last class we saw how Julia can generate specialized code for you.

Sometimes that's not enough, and you need to write a program to explicitly generate the code needed for a specialized problem.

Julia allows us to talk in a "meta" way ("one level up"), about Julia code,  that is to "treat code as data" and manipulate it as just another object in Julia. (This is very similar to Lisp.)

## Metaprogramming with text

In [1]:
x = "y + z"
y = 7
"$x + $y + $(sqrt(2))"

"y + z + 7 + 1.4142135623730951"

In [2]:
"2$x"

"2y + z"

In [3]:
x = 7
y = 2

2

In [4]:
x + y

9

## Symbols and Expressions

One of the most basic objects in this approach are unevaluated *symbols*:

In [5]:
:a  # "the symbol a"

:a

In [6]:
:foobar

:foobar

In [7]:
typeof(:a)

Symbol

`:a` refers to the symbol `a`. We can evaluate it with the `eval` function:

In [8]:
eval(:a)

UndefVarError: UndefVarError: a not defined

`a` must be defined for this to work:

In [9]:
a = 3

3

In [10]:
eval(:a)

3

The `eval` function takes an expression and evaluates it, that is, *generates the corresponding code*

Operators and function names are also symbols:

In [11]:
:+, :sin

(:+, :sin)

In [12]:
typeof(:+)

Symbol

In [13]:
+

+ (generic function with 163 methods)

Symbols may be combined into *expressions*, which are the basic objects that represent pieces of Julia code:

In [14]:
ex = :(a + b)  # the expression 'a + b'

:(a + b)

In [15]:
typeof(ex)

Expr

In [16]:
b = 7
eval(ex)

10

An expression is just a Julia object, so we can introspect (find out information about it):

In [17]:
dump(3+4im)

Complex{Int64}
  re: Int64 3
  im: Int64 4


In [18]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol a
    3: Symbol b


In [19]:
dump(:(+(a,b)))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol +
    2: Symbol a
    3: Symbol b


In [20]:
dump(:(foo(bar, baz)))

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol foo
    2: Symbol bar
    3: Symbol baz


In [21]:
dump(:(x = 3))

Expr
  head: Symbol =
  args: Array{Any}((2,))
    1: Symbol x
    2: Int64 3


In [ ]:
ex. # ex.<TAB>

In [22]:
ex.head

:call

In [23]:
ex.args

3-element Array{Any,1}:
 :+
 :a
 :b

The job of Julia's parser is to convert a sequence of characters into these `Expr` objects:

In [25]:
Meta.parse("a + b")

:(a + b)

More complicated expressions are represented as "abstract syntax trees" (ASTs), consisting of expressions nested inside expressions:

In [26]:
ex = :( sin(3a + 2b^2) )

:(sin(3a + 2 * b ^ 2))

In [27]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((2,))
    1: Symbol sin
    2: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol *
            2: Int64 3
            3: Symbol a
        3: Expr
          head: Symbol call
          args: Array{Any}((3,))
            1: Symbol *
            2: Int64 2
            3: Expr
              head: Symbol call
              args: Array{Any}((3,))
                1: Symbol ^
                2: Symbol b
                3: Int64 2


In [29]:
ex.args

2-element Array{Any,1}:
 :sin             
 :(3a + 2 * b ^ 2)

In [30]:
typeof(ex.args[2])

Expr

In [31]:
ex.args[2].args

3-element Array{Any,1}:
 :+          
 :(3a)       
 :(2 * b ^ 2)

In [32]:
ex.args[1] = :cos

:cos

In [33]:
ex

:(cos(3a + 2 * b ^ 2))

In [34]:
blk = quote
    println("Hello")
    3 + 4
end

quote
    #= In[34]:2 =#
    println("Hello")
    #= In[34]:3 =#
    3 + 4
end

In [35]:
eval(blk)

Hello


7

In [36]:
dump(blk)

Expr
  head: Symbol block
  args: Array{Any}((4,))
    1: LineNumberNode
      line: Int64 2
      file: Symbol In[34]
    2: Expr
      head: Symbol call
      args: Array{Any}((2,))
        1: Symbol println
        2: String "Hello"
    3: LineNumberNode
      line: Int64 3
      file: Symbol In[34]
    4: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Int64 3
        3: Int64 4


In [37]:
push!(blk.args, :(println("AFTER")))

5-element Array{Any,1}:
 :(#= In[34]:2 =#)  
 :(println("Hello"))
 :(#= In[34]:3 =#)  
 :(3 + 4)           
 :(println("AFTER"))

In [38]:
blk

quote
    #= In[34]:2 =#
    println("Hello")
    #= In[34]:3 =#
    3 + 4
    println("AFTER")
end

In [39]:
eval(blk)

Hello
AFTER


In [40]:
pushfirst!(blk.args, :(println("BEFORE")))

6-element Array{Any,1}:
 :(println("BEFORE"))
 :(#= In[34]:2 =#)   
 :(println("Hello")) 
 :(#= In[34]:3 =#)   
 :(3 + 4)            
 :(println("AFTER")) 

In [41]:
blk

quote
    println("BEFORE")
    #= In[34]:2 =#
    println("Hello")
    #= In[34]:3 =#
    3 + 4
    println("AFTER")
end

In [42]:
eval(blk)

BEFORE
Hello
AFTER


Expressions can be arbitrary Julia code that when evaluated will have side effects. For longer blocks of code, `quote...end` may be used instead of `:( ... )`

In [ ]:
ex2 = 
quote
    y = 3
    z = sin(y+1)
end

In [ ]:
eval(ex2)
y

In [ ]:
eval(ex2)
z

The full form of the abstract syntax tree in a style similar to a Lisp s-expression can be obtained using functions from the `Meta` module in `Base`:

In [43]:
Meta.show_sexpr(blk)

(:block,
  (:call, :println, "BEFORE"),
  :(#= In[34]:2 =#),
  (:call, :println, "Hello"),
  :(#= In[34]:3 =#),
  (:call, :+, 3, 4),
  (:call, :println, "AFTER")
)

Another way of seeing the structure is with `dump`:

In [ ]:
dump(ex2)

## Interpolation

Values can be *interpolated* into expressions with `$`.

In [45]:
a = :(x + y)
:(2 * $a)

:(2 * (x + y))

In [46]:
x = 2

2

In [47]:
:(x + $x)

:(x + 2)

In [48]:
x = :y

:y

In [49]:
:(x + $x)

:(x + y)

### Why not strings?

*Abstract* syntax vs. *concrete* syntax.

In [50]:
a = "x + y"

"x + y"

In [51]:
b = "z"

"z"

In [52]:
expr = "$a * z"

"x + y * z"

Even worse, what if `z == ")"`?

With `Expr` objects, every expression is well-formed and each slot has its abstract meaning.

### Splat interpolation

In [53]:
vars = [:a, :b, :c]

3-element Array{Symbol,1}:
 :a
 :b
 :c

In [55]:
ex = :(1 - f(0, $(vars...)))

:(1 - f(0, a, b, c))

In [56]:
dump(ex)

Expr
  head: Symbol call
  args: Array{Any}((3,))
    1: Symbol -
    2: Int64 1
    3: Expr
      head: Symbol call
      args: Array{Any}((5,))
        1: Symbol f
        2: Int64 0
        3: Symbol a
        4: Symbol b
        5: Symbol c


### `@eval` for interpolating on the fly

You will sometimes see `@eval expr`, which is shorthand for:

```
eval(quote
         expr
     end)
```

allowing items to be interpolated immediately before executing.

In [58]:
fnames = [ Symbol("func$i") for i=1:10]

10-element Array{Symbol,1}:
 :func1 
 :func2 
 :func3 
 :func4 
 :func5 
 :func6 
 :func7 
 :func8 
 :func9 
 :func10

In [66]:
for i = 1:length(fnames)
    @eval ($(fnames[i]))(x) = x + $i
    
    #equivalently:
    eval(quote
          $(fnames[i])(x) = x + $i
         end)
end

# equivalent to:
func1(x) = x + 1
func2(x) = x + 2
# etc.

func2 (generic function with 1 method)

In [61]:
func1(3)

4

In [62]:
func2(3)

5

In [63]:
func10(3)

13

## Macros

With the ability to think of code in terms of a data structure in the Julia language, we can now *manipulate* those data structures, allowing us to *create Julia code from within Julia*.

*Macros* provide a particular use pattern of metaprogramming: replacing one expression with another, in-place, right after parsing.

The [Julia manual](http://julia.readthedocs.org/en/latest/manual/metaprogramming/) puts it like this: 

> macros map a tuple of argument *expressions* to a returned
> *expression*

Macros are useful in several cases:

- to provide a specific notation different than what can normally be written in the language
    - e.g. https://github.com/JuliaOpt/JuMP.jl/blob/release-0.18/examples/sudoku.jl#L44
- to rearrange or delay evaluation of code
- to eliminate boilerplate (repetitive) code
- to automatically generate complex code that would be painful by hand
- to unroll loops for efficiency
- to inline code for efficiency


In [67]:
dump(Meta.parse("@foo x + y"))

Expr
  head: Symbol macrocall
  args: Array{Any}((3,))
    1: Symbol @foo
    2: LineNumberNode
      line: Int64 1
      file: Symbol none
    3: Expr
      head: Symbol call
      args: Array{Any}((3,))
        1: Symbol +
        2: Symbol x
        3: Symbol y



Macros are invoked using the `@` sign, e.g.

In [70]:
@time sin(10)

  0.142367 seconds (396.66 k allocations: 20.180 MiB, 4.31% gc time)


-0.5440211108893698

Compare to:
```jl
mytime(sin(10))
```
As an ordinary function, `mytime`, this **could not work**.

In [69]:
macroexpand(Main, :(@time sin(10)))

quote
    #= util.jl:154 =#
    local #2#stats = (Base.gc_num)()
    #= util.jl:155 =#
    local #4#elapsedtime = (Base.time_ns)()
    #= util.jl:156 =#
    local #3#val = sin(10)
    #= util.jl:157 =#
    #4#elapsedtime = (Base.time_ns)() - #4#elapsedtime
    #= util.jl:158 =#
    local #5#diff = (Base.GC_Diff)((Base.gc_num)(), #2#stats)
    #= util.jl:159 =#
    (Base.time_print)(#4#elapsedtime, (#5#diff).allocd, (#5#diff).total_time, (Base.gc_alloc_count)(#5#diff))
    #= util.jl:161 =#
    (Base.println)()
    #= util.jl:162 =#
    #3#val
end

A trivial example of defining a macro is the following, which runs whatever code it is passed two times.

In [71]:
macro twice(ex)
    quote
        $ex
        $ex
    end
end

@twice (macro with 1 method)

In [74]:
@twice println("hello")

hello
hello


In [75]:
ex = :(@twice println("hello"))

:(#= In[75]:1 =# @twice println("hello"))

In [76]:
eval(ex)

hello
hello


In [77]:
dump(ex)

Expr
  head: Symbol macrocall
  args: Array{Any}((3,))
    1: Symbol @twice
    2: LineNumberNode
      line: Int64 1
      file: Symbol In[75]
    3: Expr
      head: Symbol call
      args: Array{Any}((2,))
        1: Symbol println
        2: String "hello"


We can see what effect the macro actually has using `macroexpand`:

In [79]:
macroexpand(Main, :(@twice println("hello")))

quote
    #= In[71]:3 =#
    (Main.println)("hello")
    #= In[71]:4 =#
    (Main.println)("hello")
end

In [91]:
macro mytime(ex)
    quote
        t0 = time()
        val = $(esc(ex))
        t1 = time()
        println("$(t1-t0) seconds elapsed")
        val
    end
end

@mytime (macro with 1 method)

In [82]:
@mytime (sleep(1); "done")

1.0399909019470215 seconds elapsed


"done"

A line break terminates macro arguments, so blocks need to be passed with `begin...end`:

In [92]:
@mytime t0 = (sleep(1); 7)

1.0011420249938965 seconds elapsed


7

In [93]:
macroexpand(Main, :(@mytime t0 = (sleep(1); 7)))

quote
    #= In[91]:3 =#
    #43#t0 = (Main.time)()
    #= In[91]:4 =#
    #44#val = (t0 = begin
                sleep(1)
                #= In[93]:1 =#
                7
            end)
    #= In[91]:5 =#
    #45#t1 = (Main.time)()
    #= In[91]:6 =#
    (Main.println)("$(#45#t1 - #43#t0) seconds elapsed")
    #= In[91]:7 =#
    #44#val
end

----
**Exercise**: Define a macro `@until` that does an `until` loop.

```jl
x = 0
@until x > 3 begin
    global x = x+1
end
```

----

In [96]:
macro until(cond, body)
    quote
        $(esc(body))
        while !$(esc(cond))
            $(esc(body))
        end
    end
end

@until (macro with 1 method)

In [98]:
macro until(cond, body)
    quote
        while true
            $(esc(body))
            if $(esc(cond))
                break
            end
        end
    end
end

@until (macro with 1 method)

In [99]:
x = 0
@until x > 3 begin
    global x = x+1
end
x

4

In [100]:
x = 7
@until x > 3 begin
    println("hello")
end

hello


### Macro Hygiene

In [ ]:
macro set_x(val)
    :(x = $val)
end

In [ ]:
@set_x 10

In [ ]:
x

In [ ]:
macro set_x(val)
    :($(esc(:x)) = $val)
end

In [ ]:
@set_x 10

In [ ]:
x

### Macros for numerical performance: Horner's method

There are many interesting examples of macros in `Base`. One that is accessible is Horner's method for evaluating a polynomial:

$$p(x) = a_n x^n + a_{n-1} x^{n-1} + \cdots + a_1 x^1 + a_0$$

may be evaluated efficiently as

$$p(x) = a_0 + x(a_1 + \cdots x(a_{n-2} + \cdots + x(a_{n-1} + x a_n) \cdots ) ) $$
with only $n$ multiplications.

The obvious way to do this is with a `for` loop. But if we know the polynomial *at compile time*, this loop may be unrolled using metaprogramming. This is implemented in the `Math` module in `math.jl` in `Base`, so the name of the macro (which is not exported) is `@Base.Math.horner`.

In [102]:
# copied from base/math.jl
macro horner(x, p...)
    ex = esc(p[end])
    for i = length(p)-1:-1:1
        ex = :( $(esc(p[i])) + t * $ex )  # actually uses `muladd`
    end
    Expr(:block, :(t = $(esc(x))), ex)
end

@horner (macro with 1 method)

This is called as follows: to evaluate the polynomial $p(x) = 2 + 3x + 4x^2$ at $x=3$, we do

In [103]:
x = 3
@horner(x, 2, 3, 4, 5)

182

To see what the macro does to this call, we again use `macroexpand`:

In [105]:
macroexpand(Main, :(@horner(x, 2, 3, 4, 5, 6, 7, 8, 9, 10)))

quote
    #48#t = x
    2 + #48#t * (3 + #48#t * (4 + #48#t * (5 + #48#t * (6 + #48#t * (7 + #48#t * (8 + #48#t * (9 + #48#t * 10)))))))
end

In [106]:
macroexpand(Main, :(@Base.Math.horner(x, 2, 3, 4, 5)))

quote
    #49#t = x
    (Base.Math.muladd)(#49#t, (Base.Math.muladd)(#49#t, (Base.Math.muladd)(#49#t, 5, 4), 3), 2)
end

In [ ]:
?muladd

In [ ]:
?fma

In [107]:
f(x) = Base.Math.@horner(x, 1.2, 2.3, 3.4, 4.5)

f (generic function with 1 method)

In [108]:
@code_llvm f(0.1)


; Function f
; Location: In[107]:1
define double @julia_f_36037(double) {
top:
; Function muladd; {
; Location: float.jl:404
  %1 = fmul contract double %0, 4.500000e+00
  %2 = fadd contract double %1, 3.400000e+00
  %3 = fmul contract double %2, %0
  %4 = fadd contract double %3, 2.300000e+00
  %5 = fmul contract double %4, %0
  %6 = fadd contract double %5, 1.200000e+00
;}
  ret double %6
}


In [109]:
@code_native f(0.1)

	.section	__TEXT,__text,regular,pure_instructions
; Function f {
; Location: In[107]:1
	decl	%eax
	movl	$592529504, %eax        ## imm = 0x23514860
	addl	%eax, (%eax)
	addb	%al, (%eax)
; Function muladd; {
; Location: float.jl:404
	vmovsd	(%eax), %xmm1           ## xmm1 = mem[0],zero
	decl	%eax
	movl	$592529512, %eax        ## imm = 0x23514868
	addl	%eax, (%eax)
	addb	%al, (%eax)
	vfmadd213sd	(%eax), %xmm0, %xmm1
	decl	%eax
	movl	$592529520, %eax        ## imm = 0x23514870
	addl	%eax, (%eax)
	addb	%al, (%eax)
	vfmadd213sd	(%eax), %xmm0, %xmm1
	decl	%eax
	movl	$592529528, %eax        ## imm = 0x23514878
	addl	%eax, (%eax)
	addb	%al, (%eax)
	vfmadd213sd	(%eax), %xmm1, %xmm0
;}
	retl
	nopl	(%eax)
;}


In [110]:
f(x) = @evalpoly(x, 1.2, 2.3, 3.4, 4.5)

f (generic function with 1 method)

In [111]:
?@evalpoly

```
@evalpoly(z, c...)
```

Evaluate the polynomial $\sum_k c[k] z^{k-1}$ for the coefficients `c[1]`, `c[2]`, ...; that is, the coefficients are given in ascending order by power of `z`.  This macro expands to efficient inline code that uses either Horner's method or, for complex `z`, a more efficient Goertzel-like algorithm.

# Examples

```jldoctest
julia> @evalpoly(3, 1, 0, 1)
10

julia> @evalpoly(2, 1, 0, 1)
5

julia> @evalpoly(2, 1, 1, 1)
7
```


In [112]:
@code_llvm f(0.1)


; Function f
; Location: In[110]:1
define double @julia_f_36122(double) {
top:
; Function macro expansion; {
; Location: math.jl:150
; Function muladd; {
; Location: float.jl:404
  %1 = fmul contract double %0, 4.500000e+00
  %2 = fadd contract double %1, 3.400000e+00
  %3 = fmul contract double %2, %0
  %4 = fadd contract double %3, 2.300000e+00
  %5 = fmul contract double %4, %0
  %6 = fadd contract double %5, 1.200000e+00
;}}
  ret double %6
}


In [113]:
@code_llvm f(0.1+0.3im)


; Function f
; Location: In[110]:1
define void @julia_f_36126({ double, double }* noalias nocapture sret, { double, double } addrspace(11)* nocapture nonnull readonly dereferenceable(16)) {
top:
; Function macro expansion; {
; Location: math.jl:150
; Function real; {
; Location: complex.jl:63
; Function getproperty; {
; Location: sysimg.jl:18
  %2 = getelementptr inbounds { double, double }, { double, double } addrspace(11)* %1, i64 0, i32 0
;}}
; Function imag; {
; Location: complex.jl:76
; Function getproperty; {
; Location: sysimg.jl:18
  %3 = getelementptr inbounds { double, double }, { double, double } addrspace(11)* %1, i64 0, i32 1
;}}
; Function +; {
; Location: float.jl:395
  %4 = load double, double addrspace(11)* %2, align 8
  %5 = fadd double %4, %4
;}
; Function *; {
; Location: float.jl:399
  %6 = load double, double addrspace(11)* %3, align 8
  %7 = fmul double %6, %6
;}
; Function muladd; {
; Location: float.jl:404
  %8 = fmul contract double %4, %4
  %9 = fadd contrac

In [117]:
g(x) = Base.Math.@horner(x, 1.2, 2.3, 3.4, 4.5)

g (generic function with 1 method)

In [119]:
using BenchmarkTools

In [121]:
@btime f(3.0+4.0im)
@btime g(3.0+4.0im)

  1.894 ns (0 allocations: 0 bytes)
  2.261 ns (0 allocations: 0 bytes)


-542.2 + 288.79999999999995im

In [114]:
function horner(x, p)
    y = p[end]
    for i = length(p)-1:-1:1
        y = p[i] + x * y
    end
    return y
end

horner (generic function with 1 method)

In [115]:
horner(4, [3,4,5])

99

In [116]:
@horner 4 3 4 5

99

## `@generated` Functions

Another form of metaprogramming available in Julia.

Generate code based on *types* instead of the input expressions.

In [ ]:
@generated function nloops(a::Array{T,N}) where {T,N}
    # inside here,
    # - `a` refers to the type of the argument
    # - `T` and `N` have their usual values
    # - return an expression to compute the answer for this type
end